In [1]:
import numpy as np
import pandas as pd
import torch
from torch import tensor, sigmoid, mm

# Gradient

In [2]:
x = tensor(8.0, requires_grad=True)
y = (x-5) ** 2
y

tensor(9., grad_fn=<PowBackward0>)

In [3]:
y.backward()
x.grad

tensor(6.)

# Descent

In [4]:
x = tensor(8.0, requires_grad=True)
for i in range(15):
    y = (x-5) ** 2
    y.backward()
    with torch.no_grad():
        print(round(float(x), 3), x.grad)
        x -= x.grad * 0.1
        x.grad.zero_()
x

8.0 tensor(6.)
7.4 tensor(4.8000)
6.92 tensor(3.8400)
6.536 tensor(3.0720)
6.229 tensor(2.4576)
5.983 tensor(1.9661)
5.786 tensor(1.5729)
5.629 tensor(1.2583)
5.503 tensor(1.0066)
5.403 tensor(0.8053)
5.322 tensor(0.6442)
5.258 tensor(0.5154)
5.206 tensor(0.4123)
5.165 tensor(0.3299)
5.132 tensor(0.2639)


tensor(5.1056, requires_grad=True)

# Example 2: Diabetes

In [5]:
df = pd.read_csv("diabetes.csv")
df.head()

,pregnancies,glucose,bp,skin,insulin,bmi,pedigree,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
X = df.iloc[:, :-1].values
X -= X.mean(axis=0)
X /= X.std(axis=0)
np.concatenate([X, np.ones(X.shape[0]).reshape(-1, 1)], axis=1)

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.46849198,
         1.4259954 ,  1.        ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.36506078,
        -0.19067191,  1.        ],
       [ 1.23388019,  1.94372388, -0.26394125, ...,  0.60439732,
        -0.10558415,  1.        ],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.68519336,
        -0.27575966,  1.        ],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.37110101,
         1.17073215,  1.        ],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.47378505,
        -0.87137393,  1.        ]])

In [7]:
X = tensor(X, dtype=torch.float32)
y = tensor(df.iloc[:, -1].values, dtype=torch.float32)

In [8]:
A = tensor(np.random.normal(size=(1, X.shape[1])), dtype=torch.float, requires_grad=True)
A

tensor([[-0.6886,  0.7149, -0.8651,  0.8087,  0.6591,  1.4141, -1.0567, -1.6358]],
       requires_grad=True)

In [9]:
def accuracy(A, X, y):
    predicted = (sigmoid(mm(A, X.T))[0] - 0.5).ceil()
    correct = 0
    for i in range(len(y)):
        if predicted[i] == y[i]:
            correct += 1
    return correct / len(y)
accuracy(A, X, y)

0.4752604166666667

In [10]:
for i in range(50):
    loss = ((sigmoid(mm(A, X.T))[0] - y) ** 2).mean()
    if i % 10 == 0:
        print("LOSS", float(loss))
    loss.backward()
    with torch.no_grad():
        A -= A.grad * 0.1
        A.grad.zero_()

LOSS 0.3989822566509247
LOSS 0.3881254196166992
LOSS 0.37732815742492676
LOSS 0.366690993309021
LOSS 0.3563186228275299


In [11]:
accuracy(A, X, y)

0.5299479166666666